In [221]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec

# The first part, try to load jason file (not really useful, dont run it)

In [2]:
# list all files under folder 
import os
directory = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent'
for filename in os.listdir(directory):
    print(filename)

BuzzFeed_Fake_91-Webpage.json
BuzzFeed_Fake_69-Webpage.json
BuzzFeed_Fake_83-Webpage.json
BuzzFeed_Fake_29-Webpage.json
BuzzFeed_Fake_84-Webpage.json
BuzzFeed_Fake_18-Webpage.json
BuzzFeed_Fake_58-Webpage.json
BuzzFeed_Fake_85-Webpage.json
.DS_Store
BuzzFeed_Fake_28-Webpage.json
BuzzFeed_Fake_82-Webpage.json
BuzzFeed_Fake_68-Webpage.json
BuzzFeed_Fake_90-Webpage.json
BuzzFeed_Fake_59-Webpage.json
BuzzFeed_Fake_19-Webpage.json
BuzzFeed_Fake_13-Webpage.json
BuzzFeed_Fake_1-Webpage.json
BuzzFeed_Fake_46-Webpage.json
BuzzFeed_Fake_14-Webpage.json
BuzzFeed_Fake_41-Webpage.json
BuzzFeed_Fake_6-Webpage.json
BuzzFeed_Fake_54-Webpage.json
BuzzFeed_Fake_53-Webpage.json
BuzzFeed_Fake_30-Webpage.json
BuzzFeed_Fake_65-Webpage.json
BuzzFeed_Fake_37-Webpage.json
BuzzFeed_Fake_62-Webpage.json
BuzzFeed_Fake_88-Webpage.json
BuzzFeed_Fake_22-Webpage.json
BuzzFeed_Fake_77-Webpage.json
BuzzFeed_Fake_25-Webpage.json
BuzzFeed_Fake_70-Webpage.json
BuzzFeed_Fake_52-Webpage.json
BuzzFeed_Fake_55-Webpage.json
Bu

In [3]:
import json
#data=json.loads('BuzzFeed_Fake_1-Webpage.json')
path = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent/BuzzFeed_Fake_88-Webpage.json'
with open(path, 'r') as myfile:
    if len(myfile.readlines()) != 0:
        myfile.seek(0)
        data = json.load(myfile)

In [69]:
print(type(data))

<class 'dict'>


In [70]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()

In [71]:
df.head()

,top_img,text,authors,keywords,meta_data,canonical_link,images,title,url,summary,movies,publish_date,source
0,http://rightwingnews.com/wp-content/uploads/20...,Hillary’s TOP Donor Country Just Auctioned Off...,[Terresa Monroe-hamilton],[],"{'googlebot': 'noimageindex', 'og': {'site_nam...",http://rightwingnews.com/hillary-clinton-2/hil...,[http://1.gravatar.com/avatar/d35b77ff6c390071...,Hillary’s TOP Donor Country Just Auctioned Off...,http://rightwingnews.com/hillary-clinton-2/hil...,,[],{'$date': 1474912025000},http://rightwingnews.com


# read all files under same folder to dataframe (Start to Run)

below is the code to read all json file under a folder together Fake[BuzzFeed] 


In [222]:
import os
import json
myList = []
directory = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent'
file_names = os.listdir(directory)
cleaned_file_name = [f for f in file_names if f.endswith('.json')]
#for i, filename in zip(range(1),cleaned_file_name):
for filename in cleaned_file_name:   
    with open(os.path.join(directory,filename),'r',encoding="utf8") as myfile:
        #print(filename)
        data = json.load(myfile) 
        myList.append(data)
    #with open(directory + '/' + filename,'r' ,encoding="utf8", errors='ignore') as myfile:
    #    if len(myfile.readlines()) != 0:
    #        myfile.seek(0)
    #        data = json.load(myfile)
    #        print(data)
    #        #data.append(json.load(myfile))
    #       #my_df = my_df.append(pd.DataFrame.from_dict(data, orient='index'))

In [180]:
# myList is a list of dictionary 
print(len(myList))
print(len(myList[0])) # there are 13 element in the dictionary
#print(myList)
print(len(myList[0]['text'])) # in the first fake news from buzzfeed, there is 2529 words
print(len(myList[90]['text'])) # in the #91 fake news from buzzfeed, there is 1662 words

91
13
2529
1662


below is the code to read all json file under a folder together Real[BuzzFeed]

In [181]:
myList_real = []
my_df_real  = pd.DataFrame()
directory_real = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/RealNewsContent'
file_names_real = os.listdir(directory_real)
cleaned_file_name_real = [f for f in file_names_real if f.endswith('.json')]
#for i, filename in zip(range(1),cleaned_file_name):
for filename in cleaned_file_name_real:   
    with open(os.path.join(directory_real,filename),'r',encoding="utf8") as myfile:
        #print(filename)
        data = json.load(myfile) 
        myList_real.append(data)

In [182]:
print(len(myList_real))
print(len(myList_real[0]['text'])) # in the first real news from buzzfeed, there is 9903 words
print(len(myList_real[90]['text'])) # in the #91 real news from buzzfeed, there is 1627 words

91
9903
1627


The function to load glove model, the model includes 400,000 words and put these words in a dictionary Model, the dictionary like --- "a", [a vector of number]

In [183]:
# function load glove model
file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
      
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        if (word == "rahami"):
            print("rahami exist in the list in this case") # rahamim exist
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model
          
model= loadGloveModel(file)

Loading Glove Model
Done. 400000  words loaded!


checkword is a function to check if a word in the news exist in the model, if model dosent contains this word, then return true --- in the next step, if dosen't contain, then make this word to a 0 vector 

In [184]:
# check if a word appears in the glove.6B.50d.txt or not , if not return true
def checkword (search_word):
    if search_word not in model:
        return (True)

In [185]:
print(checkword("rahami")) # rahami not exist in the glove dictionary(model), should return true

True


In [187]:
import re 
news_text = []
file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"
#print(len(myList))
for i in range (0,len(myList)): # i from 0 to 90
    per_new_text= []
    #print(i)
    count = 0
    new = myList[i]
    news_content = new['text']
    new_content_token = re.sub(r'[^\w]', ' ', news_content).lower().split()
    new_content_token = [ word for word in new_content_token if word not in stopwords.words('english')]
    for word in new_content_token:
        count = count + 1
        if (checkword(word) == True): # if a word not appears in the glove txt, then set the vector to 0 
            per_word_vector = np.empty(50)
            per_word_vector.fill(0) 
          #  per_word_vector_a = np.zeros(shape=(1,50))
            per_new_text.append(per_word_vector)
            
        else:
            per_word_vector_exist= model[word]
           # arr = np.vstack((per_word_vector_exist, per_word_vector))
            per_new_text.append(per_word_vector_exist)
    per_new_text_np =  np.array(per_new_text)
    #print(per_new_text_np.shape)
    #print('the length of each news is:', len(per_new_text))
    #print(i, count)
    news_text.append(per_new_text_np) 
    
    

In [188]:
print(len(news_text)) #-- 91
print(len(news_text[0])) # 232, it is correct 
print(len(news_text[90])) # 160, it is correct, it is the rows of the matrix 
print((len(news_text[90][0]))) # 50, it is correct, so the matrix of the last news is 160 * 50, it is column of matrix
print((news_text[90][1]))
print((news_text[90][35]))

#print(news_text[90].shape)


91
232
160
50
[-0.56384    0.019917  -0.1529    -0.29327   -0.048601   0.83398
 -0.7621    -0.40209   -0.15702    0.54327   -0.18985   -0.44826
  0.48471    1.0031    -0.67926   -0.88869    1.6547     0.38999
 -0.069386  -0.20469   -0.57703    0.78333    0.094052   0.13694
  0.85131   -0.58128    0.055574  -0.38983   -0.66906    1.0201
  0.12585   -0.70496   -0.90575   -0.17205   -0.97493   -0.80834
 -0.88955   -0.13485    0.1594    -0.048676   0.13161   -0.089399
 -0.14315    0.068304   1.3964    -0.51668   -1.1126     0.0098844
  0.7708     0.02426  ]
[-1.2328e+00 -1.1042e-01 -7.7951e-01 -5.0614e-01 -6.8150e-02  6.9457e-01
  1.8967e-01 -7.6766e-01  5.4416e-03 -3.6045e-01 -1.6309e-01  5.9114e-01
 -1.4221e-01 -3.3803e-01  2.2356e-01  1.5343e-03 -1.2043e-02 -5.9315e-01
  6.2070e-01  8.7477e-02  7.2180e-01  3.1269e-01 -4.9196e-01 -1.4968e-01
 -5.2249e-01 -2.9321e+00  1.3136e-01 -1.0542e+00 -2.5628e-01 -2.7729e-01
  2.8322e+00 -8.0952e-01 -1.2425e+00 -1.3874e+00 -5.5187e-02 -4.8155e-01
  

convert BuzzFed real news to vector 

In [189]:
import re 
news_text_real = []
#print(len(myList))
for i in range (0,len(myList_real)): # i from 0 to 90
    per_new_text_real= []
    #print(i)
    new_real = myList_real[i]
    news_content_real = new_real['text']
    new_content_token_real = re.sub(r'[^\w]', ' ', news_content_real).lower().split()
    new_content_token_real = [ word for word in new_content_token_real if word not in stopwords.words('english')]
    for word in new_content_token_real:
        if (checkword(word) == True): # if a word not appears in the glove txt, then set the vector to 0 
            per_word_vector_real = np.empty(50)
            per_word_vector_real.fill(0) 
            per_new_text_real.append(per_word_vector_real)
        else:
            per_word_vector_real_exist = model[word]
            per_new_text_real.append(per_word_vector_real_exist)
    per_new_text_real_np =  np.array(per_new_text_real)
    news_text_real.append(per_new_text_real_np)

In [190]:
print((len(news_text_real))) #- 91
print((len(news_text_real[90]))) # 147 rows (words)
print(len(news_text_real[90][0]))  # 50 columns
print((news_text_real[90][1]))

91
147
50
[ 2.1942e-01  8.4548e-01 -1.1176e-01  1.5178e-01  1.0415e+00  2.7041e-01
  1.4754e-01  3.9178e-02  4.1417e-01  5.0048e-01 -5.4793e-02  3.8730e-01
  3.6538e-01 -1.7263e-01  2.9504e-01  2.4473e-01  3.9652e-01 -3.3309e-01
  2.0765e-01 -6.7227e-01 -6.2525e-02  1.1222e-01 -3.1721e-01  1.2673e-01
  3.7021e-01 -1.3004e+00 -4.3385e-01  2.3674e-03  1.6806e-01 -3.1099e-01
  2.6524e+00  2.5751e-01 -6.0634e-01 -2.4706e-01  8.8449e-02 -2.5156e-01
 -1.2524e-02 -1.2822e-01 -3.0642e-02 -8.4447e-01  7.0279e-01  2.9554e-01
 -1.9216e-01 -1.4732e-01 -4.6286e-01  2.0464e-01  4.5651e-01  4.8463e-01
  5.6849e-02  4.7113e-01]


# read all json file from politiFact folder

In [191]:
# read fake news
import os
import json
myList_fake = []
my_df  = pd.DataFrame()
directory_fake = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/PolitiFact/FakeNewsContent'
file_names_fake = os.listdir(directory_fake)
cleaned_file_name_fake = [f for f in file_names_fake if f.endswith('.json')]
#for i, filename in zip(range(1),cleaned_file_name):
for filename in cleaned_file_name_fake:   
    with open(os.path.join(directory_fake,filename),'r',encoding="utf8") as myfile:
        #print(filename)
        data = json.load(myfile) 
        myList_fake.append(data)

In [192]:
# read good news
myList_real_p = []
directory_real_p = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/PolitiFact/RealNewsContent'
file_names_real_p = os.listdir(directory_real_p)
cleaned_file_name_real_p = [f for f in file_names_real_p if f.endswith('.json')]
#for i, filename in zip(range(1),cleaned_file_name):
for filename in cleaned_file_name_real_p:   
    with open(os.path.join(directory_real_p,filename),'r',encoding="utf8") as myfile:
        #print(filename)
        data = json.load(myfile) 
        myList_real_p.append(data)

In [193]:
# convert fake news to vector
import re 
news_text_fake_p = []
#print(len(myList))
for i in range (0,len(myList_fake)): # i from 0 to 90
    per_new_text_fake_p= []
    new_fake_p = myList_fake[i]
    news_content_fake_p = new_fake_p['text']
    new_content_token_fake_p = re.sub(r'[^\w]', ' ', news_content_fake_p).lower().split()
    new_content_token_fake_p = [ word for word in new_content_token_fake_p if word not in stopwords.words('english')]
    for word in new_content_token_fake_p:
        if (checkword(word) == True): # if a word not appears in the glove txt, then set the vector to 0 
            per_word_vector_fake_p = np.empty(50)
            per_word_vector_fake_p.fill(0) 
            per_new_text_fake_p.append(per_word_vector_fake_p)
        else:
            per_word_vector_fake_p_exist = model[word]
            per_new_text_fake_p.append(per_word_vector_fake_p_exist)
    per_new_text_fake_p_np = np.array(per_new_text_fake_p)
    news_text_fake_p.append(per_new_text_fake_p_np)

In [194]:
print((len(news_text_fake_p))) #- 120
print((len(news_text_fake_p[90]))) # 320 rows (words)
print(len(news_text_fake_p[90][0]))  # 50 columns
print((news_text_fake_p[90][1]))

120
320
50
[-0.098793   0.26983    0.35304   -0.10727   -0.015183   0.053398
 -1.0824    -0.53005    0.0095416  0.070428  -0.08925   -0.62666
 -0.52662   -0.56571    1.8044     0.01686   -0.44871   -0.04146
 -1.1136     0.17488    0.49561   -0.38238    0.30185   -0.70675
 -0.35891   -1.5164    -0.024403  -0.54107   -0.36163    0.52803
  3.6553     0.71214   -0.16995    0.36368    0.3399    -0.48186
  0.10936    0.61428    0.15697   -0.70716   -1.2359    -0.014258
  0.095588  -0.30634   -0.49741   -0.049394  -0.16697    0.11972
 -0.37511    0.098348 ]


In [230]:
# convert good news to vector 
import re 
news_text_real_p = []
#print(len(myList))
for i in range (0,len(myList_real_p)): # i from 0 to 90
    per_new_text_real_p= []
    new_real_p = myList_real_p[i]
    news_content_real_p = new_real_p['text']
    new_content_token_real_p = re.sub(r'[^\w]', ' ', news_content_real_p).lower()
    new_content_token_real_p = [ word for word in new_content_token_real_p if word not in stopwords.words('english')]
    for word in new_content_token_real_p:
        if (checkword(word) == True): # if a word not appears in the glove txt, then set the vector to 0 
            per_word_vector_real_p = np.empty(50)
            per_word_vector_real_p.fill(0)
            per_new_text_real_p.append(per_word_vector_real_p)
        else:
            per_word_vector_real_p_exist = model[word]
            per_new_text_real_p.append(per_word_vector_real_p_exist)
    per_new_text_real_p_np = np.array(per_new_text_real_p)        
    news_text_real_p.append(per_new_text_real_p_np)

TypeError: argument of type 'Sequential' is not iterable

In [196]:
# check if reall all file and convert all wrods to vector
print((len(news_text_real_p))) #- 120
print((len(news_text_real_p[90]))) # 22 rows (words)
print(len(news_text_real_p[90][0]))  # 50 columns
print((news_text_real_p[90][1]))

120
22
50
[ 0.98883    0.70861    0.27428    0.91213   -0.59645    0.12086
 -0.30068   -0.35292    0.56912   -0.0069741 -0.39206   -0.15808
 -0.88061   -0.04057   -0.72813    0.063497  -0.084623  -0.4127
  0.66418    0.83209    0.45579    0.18651    0.2475     1.1723
 -0.51486   -1.5027    -0.026649  -0.23885    0.064742  -0.40828
  2.5616    -0.17731   -0.11163   -0.58672    0.12355    0.015031
 -0.75143    0.03784   -1.2439     0.42646    0.64407   -0.31094
  0.60189    0.40464    0.40851   -0.3528    -0.22341    0.49233
  0.56392   -1.1304   ]


# add fake news vecotr and real new vector to one list 

In [197]:
print(news_text[0].shape) # should change to 232 * 50 np array

(232, 50)


In [198]:
# change news_text ... to np.array
news_text = np.array(news_text)
news_text_fake_p = np.array(news_text_fake_p)
news_text_real = np.array(news_text_real)
news_text_real_p = np.array(news_text_real_p)

In [199]:
print(news_text.shape)
print(news_text_fake_p.shape)
print(news_text_real.shape)
print(news_text_real_p.shape)

(91,)
(120,)
(91,)
(120,)


In [200]:
total_news = []
total_news.extend(news_text) # extend fake news 
total_news.extend(news_text_fake_p) # extend fake news
total_news.extend(news_text_real) # extend real news
total_news.extend(news_text_real_p) # extend real news
total_news = np.array(total_news)

In [201]:
print(total_news.shape)
print(total_news[0].shape)

print(type(total_news[0]))
print(type(total_news[1]))
print(type(total_news[276]))

(422,)
(232, 50)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# create response variable y fake is 1 and real is 0, y is a list

In [202]:
# create a list to hold the response variable fake and real (1 and 0), the first 91 is fake and last 91 is real
fake_y = [1] * (91 + 120)
real_y = [0]* (91+120)
fake_real = []
fake_real.extend(fake_y)
fake_real.extend(real_y)
print(len(fake_real))

# one hot encode the output variable
news_Y = np.array (fake_real)
print(type(news_Y))

422
<class 'numpy.ndarray'>


# split training and testing data

In [203]:
training_set_portion =.8 # keep 80 % data for training
#LEt's create some random integer index and partition the data
number_of_examples = total_news.shape[0] # the length of total news
print('Total examples are {}'.format(number_of_examples))
np.random.seed(0) # to make sure multiple run give same result
random_index = np.random.permutation(range(number_of_examples))
training_set_size = int(number_of_examples*training_set_portion)
print('train set size is {} test set size is {}'.format(training_set_size,number_of_examples - training_set_size))

Total examples are 422
train set size is 337 test set size is 85


In [204]:
training_X = total_news[random_index[:training_set_size]]
training_y = news_Y[random_index[:training_set_size]]
test_X = total_news[random_index[training_set_size:]]
test_y = news_Y[random_index[training_set_size:]]

print('Shape of training X {} and train Y {}'.format(training_X.shape, training_y.shape))
print('Shape of test X {} and test Y {}'.format(test_X.shape, test_y.shape))

Shape of training X (337,) and train Y (337,)
Shape of test X (85,) and test Y (85,)


In [213]:
print(type(total_news[276]))
print(total_news[276])

print(type(training_X))
print(type(training_X[0]))
print(type(training_X[10]))
print(type(training_X[121]))
print(type(training_X[336]))

<class 'numpy.ndarray'>
[[ 0.3642    0.13178  -0.2072   ...  0.93954  -0.26746  -0.26255 ]
 [ 0.70252   0.16953   0.81731  ... -0.56448   0.14295  -0.65182 ]
 [ 2.1611   -0.22174   0.3826   ...  0.61004   0.58623  -0.70055 ]
 ...
 [ 0.41619  -0.2509   -0.4395   ...  1.1149   -0.23315   0.054302]
 [-0.42711   0.25074  -0.13306  ... -0.5564   -0.22553   0.68068 ]
 [-0.17103   0.21703   0.31748  ...  0.64138   0.36659   0.41078 ]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Model

In [5]:
# LSTM with Variable Length Input Sequences to One Character Output
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
#from theano.tensor.shared_randomstreams import RandomStreams

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [227]:
docs = [item['text'] for item in myList]

In [235]:
#@TODO replace with cleaned trext
docs[0]

"People Noticed Something Odd About Hillary’s Outfit at the Debate Last Night\n\nThere’s a lot that could be discussed about last night’s debate, like the unfair debate moderator asking questions to help Hillary, but that’s not the main concern right now.\n\nThere is one thing that has baffled many…. Hillary didn’t appear to be her usually stupid coughing self! She was so ready with answers and detailed facts. Almost just as bizarre was that not once did she sputter or reach for a glass of water. This is so odd that it has everyone and their grandma talking!\n\nThe latest question swirling around on social media are the pictures that appeared to show Hillary with some kind of flesh-colored device embedded inside her ear…is that an earpiece she is wearing? Or just something fashionable for old ladies?\n\nTwitter may have figured out the answer…do you see it?\n\nIt certainly wouldn’t be the first time Hillary had to use a device for help. What do you think? Could it be a trick of light a

In [6]:
docs = ['Well done!','Good work',
'Great effort',
'nice work',
'Excellent!']

In [7]:
docs

['Well done!', 'Good work', 'Great effort', 'nice work', 'Excellent!']

In [8]:
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)


In [9]:
t.word_index

{'done': 3,
 'effort': 6,
 'excellent': 8,
 'good': 4,
 'great': 5,
 'nice': 7,
 'well': 2,
 'work': 1}

In [10]:
examples_int_emb  = t.texts_to_sequences(docs)

In [11]:
examples_int_emb

[[2, 3], [4, 1], [5, 6], [7, 1], [8]]

In [13]:
pad_sequences(examples_int_emb, maxlen=2)

array([[2, 3],
       [4, 1],
       [5, 6],
       [7, 1],
       [0, 8]], dtype=int32)

In [ ]:
from keras_preprocessing import 

In [214]:
batch_size = 1
model = Sequential()
model.add(LSTM(32, input_shape=(None,50)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(training_X, training_y, epochs=500, batch_size=batch_size, verbose=2)


ValueError: Error when checking input: expected lstm_2_input to have 3 dimensions, but got array with shape (337, 1)

# LSTM with Variable-Length Input to One-Char Output

In [215]:
# LSTM with Variable Length Input Sequences to One Character Output
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from theano.tensor.shared_randomstreams import RandomStreams

In [216]:
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
num_inputs = 50
max_len = 5
dataX = []
dataY = []
for i in range(num_inputs):
    start = numpy.random.randint(len(alphabet)-2)
    end = numpy.random.randint(start, min(start+max_len,len(alphabet)-1))
    sequence_in = alphabet[start:end+1]
    sequence_out = alphabet[end + 1]
    dataX.append([char_to_int[char] for char in sequence_in])
    dataY.append(char_to_int[sequence_out])
    print (sequence_in, '->', sequence_out)
# convert list of lists to array and pad sequences if needed
X = pad_sequences(dataX, maxlen=max_len, dtype='float32')
# reshape X to be [samples, time steps, features]
X = numpy.reshape(X, (X.shape[0], max_len, 1))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# create and fit the model
batch_size = 1
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], 1)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=batch_size, verbose=2)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))
# demonstrate some model predictions
for i in range(20):
    pattern_index = numpy.random.randint(len(dataX))
    pattern = dataX[pattern_index]
    x = pad_sequences([pattern], maxlen=max_len, dtype='float32')
    x = numpy.reshape(x, (1, max_len, 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print (seq_in, "->", result)

PQRST -> U
W -> X
O -> P
OPQ -> R
IJKLM -> N
QRSTU -> V
ABCD -> E
X -> Y
GHIJ -> K
M -> N
XY -> Z
QRST -> U
ABC -> D
JKLMN -> O
OP -> Q
XY -> Z
D -> E
T -> U
B -> C
QRSTU -> V
HIJ -> K
JKLM -> N
ABCDE -> F
X -> Y
V -> W
DE -> F
DEFG -> H
BCDE -> F
EFGH -> I
BCDE -> F
FG -> H
RST -> U
TUV -> W
STUV -> W
LMN -> O
P -> Q
MNOP -> Q
JK -> L
MNOP -> Q
OPQRS -> T
UVWXY -> Z
PQRS -> T
D -> E
EFGH -> I
IJK -> L
WX -> Y
STUV -> W
MNOPQ -> R
P -> Q
WXY -> Z
Epoch 1/500
 - 1s - loss: 3.2631 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.2132 - acc: 0.0800
Epoch 3/500
 - 0s - loss: 3.1571 - acc: 0.0800
Epoch 4/500
 - 0s - loss: 3.0630 - acc: 0.0800
Epoch 5/500
 - 0s - loss: 2.9684 - acc: 0.0800
Epoch 6/500
 - 0s - loss: 2.9127 - acc: 0.0800
Epoch 7/500
 - 0s - loss: 2.8872 - acc: 0.0800
Epoch 8/500
 - 0s - loss: 2.8627 - acc: 0.0800
Epoch 9/500
 - 0s - loss: 2.8427 - acc: 0.0800
Epoch 10/500
 - 0s - loss: 2.8242 - acc: 0.1000
Epoch 11/500
 - 0s - loss: 2.7972 - acc: 0.0800
Epoch 12/500
 - 0s - loss:

KeyboardInterrupt: 

In [218]:
print(X.shape[1])

5


# Some examples or some useless code 

In [49]:
# example of the content of the first news
# from the list of dictionary , delete all \n or \nn for one new
news_text = []
new = myList[0]
news_content = new['text']
result = []
#print(news_content) # sentence
new_content_token= news_content.replace('\n', ' ').lower().split()
#print(new_content_token) # a list of words
# remove stopwords word like I , am , my, the ...
new_content_token = [ word for word in new_content_token if word not in stopwords.words('english')]
#print(new_content_token)
# remove wired charactor like ?, ..., "', multiple X, 101, #, :, it's, we'll
for word in new_content_token: 
    sc = "….?’,“”'#!×:"
    for char in sc:
        word = word.replace(char," ")  
    word = word.split()
    #print(word)
    result.extend(word)
print(result)
for element in result:
    #print(element)
    per_word_vector = model[element]
    
    #print(model[element])
    #print(type(model[element])) # type of per_word_vector is numpy.ndarray 
    
    
# convert each word in new_content_token to vector using word2vec or glove vector
#print(model['hello'])
#print(len(model['hello']))
#for word in new_content_token:
    #print(model[word])
    

['people', 'noticed', 'something', 'odd', 'hillary', 's', 'outfit', 'debate', 'last', 'night', 'there', 's', 'lot', 'could', 'discussed', 'last', 'night', 's', 'debate', 'like', 'unfair', 'debate', 'moderator', 'asking', 'questions', 'help', 'hillary', 'that', 's', 'main', 'concern', 'right', 'now', 'one', 'thing', 'baffled', 'many', 'hillary', 'didn', 't', 'appear', 'usually', 'stupid', 'coughing', 'self', 'ready', 'answers', 'detailed', 'facts', 'almost', 'bizarre', 'sputter', 'reach', 'glass', 'water', 'odd', 'everyone', 'grandma', 'talking', 'latest', 'question', 'swirling', 'around', 'social', 'media', 'pictures', 'appeared', 'show', 'hillary', 'kind', 'flesh-colored', 'device', 'embedded', 'inside', 'ear', 'is', 'earpiece', 'wearing', 'something', 'fashionable', 'old', 'ladies', 'twitter', 'may', 'figured', 'answer', 'do', 'see', 'it', 'certainly', 'wouldn', 't', 'first', 'time', 'hillary', 'use', 'device', 'help', 'think', 'could', 'trick', 'light', 'shadows', 'puffy', 'zipper',

In [7]:
print(type(model[element]))

<class 'numpy.ndarray'>


In [16]:
# example to delete ' and s
a = 'hillary’s'
b = "’s"
for char in b:
    a = a.replace(char,"")
    print(a)

hillarys
hillary


In [13]:
#print(new_content_token)
print(type(new_content_token))

<class 'list'>
people


In [58]:
data.keys()

dict_keys(['top_img', 'text', 'authors', 'keywords', 'meta_data', 'canonical_link', 'images', 'title', 'url', 'summary', 'movies', 'publish_date', 'source'])

In [56]:
# concatenate string
new = directory + '/'+ 'BuzzFeed_Fake_80-Webpage.json'
print(new)

/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent/BuzzFeed_Fake_80-Webpage.json


# Example1 : convert text to vector Example

In [2]:
#word2vec example1/ using own word embeddings
from gensim.models import Word2Vec

# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for!', 'word2vec'],
             ['this', 'is', 'the', 'second', 'sentence'],
             ['yet', 'another', 'sentence'],
             ['one', 'more', 'sentence'],
             ['and', 'the', 'final', 'sentence']]
# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print('model is ',model)
# summarize vocabulary,all words that appears in the sentences
words = list(model.wv.vocab)
print('words is ',words)
# access vector for one word
print('access vecotr for one word', model['sentence'])
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

from sklearn.decomposition import PCA
from matplotlib import pyplot
# plot 
X = model[model.wv.vocab]

pca = PCA(n_components=2)
result = pca.fit_transform(X)

pyplot.scatter(result[:, 0], result[:, 1])

words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

model is  Word2Vec(vocab=14, size=100, alpha=0.025)
words is  ['this', 'is', 'the', 'first', 'sentence', 'for!', 'word2vec', 'second', 'yet', 'another', 'one', 'more', 'and', 'final']
access vecotr for one word [ 4.14259126e-03  2.79425620e-03 -2.77865003e-03 -2.06513098e-03
 -3.35403159e-03  2.78954208e-03  3.93661810e-03  1.55871559e-03
 -1.44100119e-03 -4.32085153e-03  4.30899672e-03 -4.62976005e-03
 -3.79096786e-03  2.00172933e-03 -4.60096542e-03 -1.31963647e-03
  2.37290107e-04  1.49146968e-03  2.41897837e-03  2.93111708e-03
  3.15664662e-03 -2.23306101e-03 -5.75355079e-04  6.63872808e-04
  4.13089339e-03  6.29035581e-04  1.40708033e-03 -6.88619097e-04
  4.30145860e-03 -1.68555172e-03  2.99498509e-03  3.51108215e-03
 -1.11267739e-03  2.78331194e-04  3.57962656e-03  1.52858009e-03
  1.35704526e-03  4.27913677e-04 -3.05608939e-03 -3.82844337e-05
  3.61788506e-03 -3.90263507e-03  3.21124983e-03  3.75231705e-03
 -1.91865454e-03  3.26324417e-03 -2.85139750e-03  4.45809821e-03
 -3.87078

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


# Example2: golve / not run this code again

In [5]:
file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    
     
    with open(gloveFile, encoding="utf8" ) as f:
       content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model
     
     
model= loadGloveModel(file)   
 
print (model['hello'])

Loading Glove Model
Done. 400000  words loaded!
[-0.38497   0.80092   0.064106 -0.28355  -0.026759 -0.34532  -0.64253
 -0.11729  -0.33257   0.55243  -0.087813  0.9035    0.47102   0.56657
  0.6985   -0.35229  -0.86542   0.90573   0.03576  -0.071705 -0.12327
  0.54923   0.47005   0.35572   1.2611   -0.67581  -0.94983   0.68666
  0.3871   -1.3492    0.63512   0.46416  -0.48814   0.83827  -0.9246
 -0.33722   0.53741  -1.0616   -0.081403 -0.67111   0.30923  -0.3923
 -0.55002  -0.68827   0.58049  -0.11626   0.013139 -0.57654   0.048833
  0.67204 ]


# Example of using  LSTM Recurrent Neural Networks in Python with Keras

Learn the Alphabet

In [93]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [94]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [97]:
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
print(char_to_int)
print(int_to_char)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


In [101]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    print(seq_in, '->', seq_out)
print(dataX)
print(dataY)

A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24]]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [103]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

[[[ 0]]

 [[ 1]]

 [[ 2]]

 [[ 3]]

 [[ 4]]

 [[ 5]]

 [[ 6]]

 [[ 7]]

 [[ 8]]

 [[ 9]]

 [[10]]

 [[11]]

 [[12]]

 [[13]]

 [[14]]

 [[15]]

 [[16]]

 [[17]]

 [[18]]

 [[19]]

 [[20]]

 [[21]]

 [[22]]

 [[23]]

 [[24]]]


In [107]:
# normalize
X = X / float(len(alphabet))
print(X)

[[[0.        ]]

 [[0.00147929]]

 [[0.00295858]]

 [[0.00443787]]

 [[0.00591716]]

 [[0.00739645]]

 [[0.00887574]]

 [[0.01035503]]

 [[0.01183432]]

 [[0.01331361]]

 [[0.0147929 ]]

 [[0.01627219]]

 [[0.01775148]]

 [[0.01923077]]

 [[0.02071006]]

 [[0.02218935]]

 [[0.02366864]]

 [[0.02514793]]

 [[0.02662722]]

 [[0.02810651]]

 [[0.0295858 ]]

 [[0.03106509]]

 [[0.03254438]]

 [[0.03402367]]

 [[0.03550296]]]


In [106]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0.

LSTM Models:
1. Naive LSTM for Learning One-Char to One-Char Mapping (have problem)
2. Naive LSTM for a Three-Char Feature Window to One-Char Mapping (have problem)
3. Naive LSTM for a Three-Char Time Step Window to One-Char Mapping (have problem)
4. LSTM State Within A Batch (good model) 
5. Stateful LSTM for a One-Char to One-Char Mapping
6. LSTM with Variable-Length Input to One-Char Output


In [21]:
# We will frame the problem as a random collection of one-letter input to one-letter output pairs. 
# As we will see this is a difficult framing of the problem for the LSTM to learn.

learns random subsequences of the alphabet and an effort to build a model that can be given arbitrary letters or subsequences of letters and predict the next letter in the alphabet.



PQRST -> U
W -> X
O -> P
OPQ -> R
IJKLM -> N
QRSTU -> V
ABCD -> E
X -> Y
GHIJ -> K
M -> N
XY -> Z
QRST -> U
ABC -> D
JKLMN -> O
OP -> Q
XY -> Z
D -> E
T -> U
B -> C
QRSTU -> V
HIJ -> K
JKLM -> N
ABCDE -> F
X -> Y
V -> W
DE -> F
DEFG -> H
BCDE -> F
EFGH -> I
BCDE -> F
FG -> H
RST -> U
TUV -> W
STUV -> W
LMN -> O
P -> Q
MNOP -> Q
JK -> L
MNOP -> Q
OPQRS -> T
UVWXY -> Z
PQRS -> T
D -> E
EFGH -> I
IJK -> L
WX -> Y
STUV -> W
MNOPQ -> R
P -> Q
WXY -> Z
Epoch 1/500
 - 1s - loss: 3.2632 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.2132 - acc: 0.0800
Epoch 3/500
 - 0s - loss: 3.1570 - acc: 0.0800
Epoch 4/500
 - 0s - loss: 3.0627 - acc: 0.0800
Epoch 5/500
 - 0s - loss: 2.9682 - acc: 0.0800
Epoch 6/500
 - 0s - loss: 2.9126 - acc: 0.0800
Epoch 7/500
 - 0s - loss: 2.8873 - acc: 0.0800
Epoch 8/500
 - 0s - loss: 2.8627 - acc: 0.0800
Epoch 9/500
 - 0s - loss: 2.8430 - acc: 0.0800
Epoch 10/500
 - 0s - loss: 2.8245 - acc: 0.1000
Epoch 11/500
 - 0s - loss: 2.7976 - acc: 0.0800
Epoch 12/500
 - 0s - loss:

 - 0s - loss: 1.0061 - acc: 0.7000
Epoch 162/500
 - 0s - loss: 1.0111 - acc: 0.6200
Epoch 163/500
 - 0s - loss: 1.0090 - acc: 0.6200
Epoch 164/500
 - 0s - loss: 0.9907 - acc: 0.6800
Epoch 165/500
 - 0s - loss: 0.9907 - acc: 0.6800
Epoch 166/500
 - 0s - loss: 0.9993 - acc: 0.6600
Epoch 167/500
 - 0s - loss: 0.9663 - acc: 0.6600
Epoch 168/500
 - 0s - loss: 0.9744 - acc: 0.6600
Epoch 169/500
 - 0s - loss: 0.9739 - acc: 0.6800
Epoch 170/500
 - 0s - loss: 0.9613 - acc: 0.7200
Epoch 171/500
 - 0s - loss: 0.9723 - acc: 0.6800
Epoch 172/500
 - 0s - loss: 0.9678 - acc: 0.6800
Epoch 173/500
 - 0s - loss: 0.9588 - acc: 0.6800
Epoch 174/500
 - 0s - loss: 0.9615 - acc: 0.6200
Epoch 175/500
 - 0s - loss: 0.9477 - acc: 0.6800
Epoch 176/500
 - 0s - loss: 0.9478 - acc: 0.6800
Epoch 177/500
 - 0s - loss: 0.9701 - acc: 0.6800
Epoch 178/500
 - 0s - loss: 0.9578 - acc: 0.6600
Epoch 179/500
 - 0s - loss: 0.9358 - acc: 0.7000
Epoch 180/500
 - 0s - loss: 0.9453 - acc: 0.6600
Epoch 181/500
 - 0s - loss: 0.9217

KeyboardInterrupt: 